In [3]:
import pandas as pd
import numpy as np

6.1文本格式数据的读写
由于现实世界的数据非常混乱，随着事件推移，一些数据夹杂自函数（尤其是read_csv）的可选参数变得非常复杂。pandas在线文档有大量的示例展示这些参数是如何工作的。

In [9]:
%pwd
df1 = pd.read_csv('examples/ex1.csv')
print(df1)

# 使用read_table，并指定分割符
df2 = pd.read_table('examples/ex1.csv', sep=',')
print(df2)

# 有的文件不不包含表头行，可以允许pandas自动分配默认列名，也可以自己指定列名
df3 = pd.read_csv('examples/ex2.csv', header=None)
print(df3)
df4 = pd.read_csv('examples/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])
print(df4)

# 指定某一位置的列为索引
names = ['a', 'b', 'c', 'd', 'message']
df5 = pd.read_csv('examples/ex2.csv', names=names, index_col='message')
print(df5)

# 当你需要从多个列中形成一个分层所有，需要传入一个包含序列号或列名的列表
parsed = pd.read_csv('examples/csv_mindex.csv', index_col=['key1','key2'])
print(parsed)

   a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo
   a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo
   0   1   2   3      4
0  1   2   3   4  hello
1  5   6   7   8  world
2  9  10  11  12    foo
   a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo
         a   b   c   d
message               
hello    1   2   3   4
world    5   6   7   8
foo      9  10  11  12
           value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16


In [18]:
# 当字段是以不同数量的空格分开时，可以通过向read_table传入一个正则表达式作为分隔符
result = pd.read_table('examples/ex3.txt', sep='\s+')
print(result)

# 可以使用skiprows 来跳过某些行
print(pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 3]))

# 缺失值处理
# 默认情况下，pandas使用常见的标识，例如NA和NULL
print(pd.read_csv('examples/ex5.csv'))
# na_values选项可以传入一个列表或者一组字符串来处理缺失值，即需要用NA替换的值序列
print(pd.read_csv('examples/ex5.csv', na_values=['NULL']))
# 在字典中，每列可以指定不同的缺失值标识
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
print(pd.read_csv('examples/ex5.csv',na_values=sentinels))

            A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491
   a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo
  something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       two  5   6   NaN   8   world
2     three  9  10  11.0  12     foo
  something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       two  5   6   NaN   8   world
2     three  9  10  11.0  12     foo
  something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       NaN  5   6   NaN   8   world
2     three  9  10  11.0  12     NaN


6.1.1 分块读入文本文件
当处理大型文件或找出正确的参数集来正确处理大文件时，可能需要读入文件的一个小片段或者按小块遍历文件

In [30]:
# 在尝试大文件之前，我们可以先对pandas的显示设置进行调整，使之更为紧凑
pd.options.display.max_rows = 10
result = pd.read_csv('examples/ex6.csv')
print(result)

# 读取一小部分（避免读取整个文件），可以知名nrows
print(pd.read_csv('examples/ex6.csv', nrows=5))

# 为了分块读入文件，可以指定chunksize作为每一块的行数
chunksize = pd.read_csv('examples/ex6.csv', chunksize=1000)
print(type(chunksize))
# read_csv返回的TextParser对象允许你根据chunksize遍历文件。
# 例如，便利ex6.csv，并对‘key’列聚合获得计数值
tot = pd.Series([])
for piece in chunksize:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
    
tot = tot.sort_values(ascending=False)
print(tot[:10])

           one       two     three      four key
0     0.467976 -0.038649 -0.295344 -1.824726   L
1    -0.358893  1.404453  0.704965 -0.200638   B
2    -0.501840  0.659254 -0.421691 -0.057688   G
3     0.204886  1.074134  1.388361 -0.982404   R
4     0.354628 -0.133116  0.283763 -0.837063   Q
...        ...       ...       ...       ...  ..
9995  2.311896 -0.417070 -1.409599 -0.515821   L
9996 -0.479893 -0.650419  0.745152 -0.646038   E
9997  0.523331  0.787112  0.486066  1.093156   K
9998 -0.362559  0.598894 -1.843201  0.887292   G
9999 -0.096376 -1.012999 -0.657431 -0.573315   0

[10000 rows x 5 columns]
        one       two     three      four key
0  0.467976 -0.038649 -0.295344 -1.824726   L
1 -0.358893  1.404453  0.704965 -0.200638   B
2 -0.501840  0.659254 -0.421691 -0.057688   G
3  0.204886  1.074134  1.388361 -0.982404   R
4  0.354628 -0.133116  0.283763 -0.837063   Q
<class 'pandas.io.parsers.TextFileReader'>
E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J

6.1.2 将数据写入文本格式

In [44]:
# 读取CSV文件
data = pd.read_csv('examples/ex5.csv')
print(data)

# 使用DataFrame的to_csv方法，我们可以将数据到处为逗号分隔的文件
data.to_csv('examples/out.csv')

# 使用其他的分隔符（写入到sys.stdout）
import sys
data.to_csv(sys.stdout, sep='|')

# 缺失值在输出时以空白字符串出现，可以用其他标识值对缺失值进行标注
data.to_csv(sys.stdout, na_rep='NaN')

# 如果没有其它选项被指定的话，行和列的标签都会被写入，不过二者也都可以禁止
data.to_csv(sys.stdout, index=False, header=False)

# 仅仅写入列的自己，并且按照选择的顺序写入
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

# Series也有to_csv方法
dates = pd.date_range('27/9/2018', periods=7)
ts = pd.Series(np.arange(7), index=dates)
ts.to_csv(sys.stdout)

  something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       two  5   6   NaN   8   world
2     three  9  10  11.0  12     foo
|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo
,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo
one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo
a,b,c
1,2,3.0
5,6,
9,10,11.0
2018-09-27,0
2018-09-28,1
2018-09-29,2
2018-09-30,3
2018-10-01,4
2018-10-02,5
2018-10-03,6


6.1.3 使用分割格式
绝大多数的表型数据都可以使用函数pandas.readd_table从硬盘中读取。然而，在某些情况下，一些手动操作时必不可少的。接收一个带有一行或多行错误的文件并不少见，read_table也无法解决这种情况。

In [48]:
import csv

# 将文件读取为行的列表
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))

# 将数据拆分为列名行和数据行
header, values = lines[0], lines[1:]

# 使用字典推到是和表达式zip(*values)生成一个包含数据列的字典，字典中行专制成列
data_dict = {h: v for h, v in zip(header, zip(*values))}
print(data_dict)

# CSV文件有多种不同分格。如需根据不同的分隔符、字符串引用约定或行中止符定义一种新的格式时，我们可以使用CSV_Dialect定义一个简单的子类。
class my_dialect(csv.Dialect):
    lineterminator = '\n' # 行终止符，默认'\r\n'，读取器会忽略行中止符兵识别跨平台行中止符
    delimiter = ';' # 一个用于分割字段的字符，默认是','
    quotechar = '"' # 用在含有特殊字符字段中的引号，默认是'"'
    quoting = csv.QUOTE_MINIMAL # 引用管理。选项包括csv.QUOTE_ALL(引用所有字段)，csv.QUOTE_MINIMAL(只使用特殊符号，如分隔符)，csv.QUOTE_NONNUMERIC和csv.QUOTE_NONE(不引用)。细节参考Python文档，默认是QUOTE_MINIMAL
    skipinitialspace = False # 忽略每个分隔符后的空白，默认是False
    doublequote = False # 如何处理字段内部引号。如果是True，则是双引号（完整细节和行为请参考在线文档）
    # escapechar = None # 当引用设置为csv.QUOTE_NONE时用于转义分隔符的字符串，默认禁用 

with open('examples/mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}


6.1.4 JSON数据
JSON(JavaScript Object Notation)已经成为Web浏览器和其他应用通过HTTP请求发送数据的标准格式。它是一种比CSV等表格文本形式更为自由的数据形式

In [9]:
data = pd.read_json('examples/example.json')

print(data)

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [16]:
# pandas.read_html 函数有很多选项，但是默认情况下，它会搜索并尝试解析所有包含在<table>标签中的表格型数据，返回的结果是DataFrame对象的列表
tables = pd.read_html('examples/fdic_failed_bank_list.html')
failures = tables[0]
print(failures.head())

# 数据清洗和分析工作，比如计算每年银行倒闭的数量
close_timestamps = pd.to_datetime(failures['Closing Date'])
close_timestamps.dt.year.value_counts()

                      Bank Name             City  ST   CERT  \
0                   Allied Bank         Mulberry  AR     91   
1  The Woodbury Banking Company         Woodbury  GA  11297   
2        First CornerStone Bank  King of Prussia  PA  35312   
3            Trust Company Bank          Memphis  TN   9956   
4    North Milwaukee State Bank        Milwaukee  WI  20364   

                 Acquiring Institution        Closing Date       Updated Date  
0                         Today's Bank  September 23, 2016  November 17, 2016  
1                          United Bank     August 19, 2016  November 17, 2016  
2  First-Citizens Bank & Trust Company         May 6, 2016  September 6, 2016  
3           The Bank of Fayette County      April 29, 2016  September 6, 2016  
4  First-Citizens Bank & Trust Company      March 11, 2016      June 16, 2016  


23    31
30    30
19    30
20    27
17    23
16    22
4     21
18    21
7     20
15    20
2     20
14    20
24    19
28    18
27    18
22    18
11    18
21    17
29    16
26    15
5     15
13    14
10    14
8     14
6     14
25    13
12    10
9     10
31     8
1      8
3      3
Name: Closing Date, dtype: int64

6.1.5.1 使用lxml.objectify 解析XML

In [21]:
from lxml import objectify

path = 'datasets/mta_perf/Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

data = []
skip_fields = ['INDICATOR_SEQ', 'PARENT_SEQ', 'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR: # 遍历root下的所有INDICATOR
    el_data = {}
    for child in elt.getchildren(): # 遍历INDICATOR下所有的子节点
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

perf = pd.DataFrame(data)
print(perf.head())

# XML数据可以更复杂，每个班标签也可以包含元数据。考虑一个HTML连接标签，也可以是有效的XML
from io import StringIO
tag = '<a href="http://www.google.hk">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()

print(root.get('href'))
print(root.text)

            AGENCY_NAME            CATEGORY  \
0  Metro-North Railroad  Service Indicators   
1  Metro-North Railroad  Service Indicators   
2  Metro-North Railroad  Service Indicators   
3  Metro-North Railroad  Service Indicators   
4  Metro-North Railroad  Service Indicators   

                                         DESCRIPTION FREQUENCY  \
0  Percent of commuter trains that arrive at thei...         M   
1  Percent of commuter trains that arrive at thei...         M   
2  Percent of commuter trains that arrive at thei...         M   
3  Percent of commuter trains that arrive at thei...         M   
4  Percent of commuter trains that arrive at thei...         M   

                         INDICATOR_NAME INDICATOR_UNIT MONTHLY_ACTUAL  \
0  On-Time Performance (West of Hudson)              %           96.9   
1  On-Time Performance (West of Hudson)              %             95   
2  On-Time Performance (West of Hudson)              %           96.9   
3  On-Time Performance (West

In [25]:
frame = pd.read_csv('examples/ex1.csv')
print(frame)
# pandas对象拥有一个to_pickle 方法可以将数据以pickle 格式写入硬盘
frame.to_pickle('examples/frame_pickle')

# 可以直接使用内建的pickle读取文件中的"pickle化"对象，或更方便地 使用pandas.read_pickle
print(pd.read_pickle('examples/frame_pickle'))

   a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo
   a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo


6.2.1 使用HDF5格式
HDF5是一个备受好评的文件格式，用于存储大量的科学数组数据。 HDF5适合处理不适合在内存中存储的大型数据，可以使你高校读写大型数据组的一小块。

In [27]:
# pandas 提供HDFStore 类像字典一样工作并处理低级别的细节
# 未能成功安装工具包库
frame = pd.DataFrame({'a': np.random.randn(100)})
store = pd.HDFStore('examples/mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']

print(store)

ImportError: HDFStore requires PyTables, "No module named 'tables'" problem importing

如果处理存储在远程服务器上的数据时，不如Amazon S3或HDFS，使用其他专门为分布式存储而设计的二进制格式更为合适，比如Apache Parquet。Parquet和其他类似的存储格式仍然在发展中。
如果是在本地处理大量数据，推荐尝试PyTables和h5py，看看他们是否符合你的需求。由于很多数据分析的困难在于I/O密集（而不是CPU密集），使用像HDF5这样的工具可以大大加速你的应用。

6.2.2 读取Microdoft Excel文件
pandas 也支持通过ExcelFile 类或pandas.read_excel函数来读取存储在Excel文件中的表格行数据。

In [32]:
# 使用ExcelFile
xlsx = pd.ExcelFile('examples/ex1.xlsx')

# 存储在表中的数据可以通过pandas.read_excel 读取到DataFrame
ex1 = pd.read_excel(xlsx, 'Sheet1')
print(ex1)

# 如果读取的是含有多个表的文件，生成ExcelFile更快，但你也可以更简洁地将文件名传入pandas.read_excel
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')
print(frame)

# 将pandas数据写入到Excel表格中
writer = pd.ExcelWriter('examples/ex2.xlsx')
frame.to_excel(writer, 'Sheet1')
writer.save()

# 也可以直接将文件路径传给to_excel，避免直接调用ExcelWriter
frame.to_excel('examples/ex2.xlsx')

   a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo
   a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo


6.3 与Web API交互
很多网站都有公开API，通过JSON 或其他数据格式提供数据服务。有多种方式可以利用Python 来访问API；推荐的简单易用方式是使用request包。

In [37]:
import requests

url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)

# Response(响应)对象的json方法将返回的一个包含解析为本地Python对象的JSON字典
data = resp.json()
print(data[0]['title'])

issues = pd.DataFrame(data, columns=['number', 'title', 'labels', 'state'])
print(issues)

fix quantile docstring
    number                                              title  \
0    22906                             fix quantile docstring   
1    22905  pandas/tests/indexes/datetimes/test_astype.py ...   
2    22904  DOC GH22897 Fix docstring of join in pandas/co...   
3    22903              lib.is_scalar misses PEP 3141 numbers   
4    22902  DOC: #22899, Fixed docstring of itertuples in ...   
5    22901    CLN GH22873 Replace base excepts in pandas/core   
6    22900   DOC/STYLE: Fix "F821 undefined name 'pd'" errors   
7    22899  DOC: Fix the docstring of itertuples in pandas...   
8    22898  DOC: Fix the docstring of quantile in pandas/c...   
9    22897  DOC: Fix the docstring of join in pandas/core/...   
10   22896  DOC: Fix the docstring of to_stata in pandas/c...   
11   22895  DOC: Fix the docstring of _set_axis_name in pa...   
12   22894  DOC: Fix the docstring of resample in pandas/c...   
13   22893  DOC: Fix the docstring of groupby in pandas/co...   
14

6.4 与数据库交互
在业务场景中，大部分数据并不是存储在文本或Excel文件中。基于SQL的关系型数据库（例如SQL Server、PostgreSQL 和MySQL）使用广泛。很多小众数据库也变得越发流行。数据库的选择通常取决于性能、数据完整性以及应用的可伸缩性需求。

In [39]:
import sqlite3

# 使用python内建的sqlite3 驱动来生成一个SQLite 数据库
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
c REAL, d INTEGER 
);
"""
con = sqlite3.connect('mydata.sqlite')
con.execute(query)
con.commit()

# 再插入几行数据
data = [('Atlanta', 'Georgia', 1.25, 6),
       ('Yallahassee', 'Florida', 2.6, 3),
       ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()

In [42]:
# 从数据库表中选择数据，大部分Python的SQL驱动（PyODBC、psycopg2、MySQLdb、pymssql等）返回的是元祖列表
cursor = con.execute('select * from test')
rows = cursor.fetchall()
print(rows)

# 可以将远足的列表传给DataFrame 构造函数，但需要包含在游标的description属性中的列名
print(cursor.description)
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

[('Atlanta', 'Georgia', 1.25, 6), ('Yallahassee', 'Florida', 2.6, 3), ('Sacramento', 'California', 1.7, 5)]
(('a', None, None, None, None, None, None), ('b', None, None, None, None, None, None), ('c', None, None, None, None, None, None), ('d', None, None, None, None, None, None))


,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Yallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


由于内容较多，你肯定不想每次查询数据库都重复同样多的步骤。SQLAlchemy 项目是一个流行的Python SQL工具包，抽象去除了SQL数据库之间的许多常见差异。pandas有一个read_sql函数允许你从通过SQLAlchemy 连接中轻松地读取数据。这里，我将使用SQLAlechemy连接到相同的SQLite数据库，并从之前创建的表中读取数据。

In [4]:
import sqlalchemy as sqla

db = sqla.create_engine('sqlite:///mydata.sqlite')
pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Yallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
